In [ ]:
import pandas as pd
# from bs4 import BeautifulSoup
# import numpy as np
import requests
# import re



# Precip Data

### Scraping data 

In [ ]:
url = "https://www.ncei.noaa.gov/pub/data/cirs/drd/drd964x.pdsi.txt"


#picking names for files to save. both a txt and a csv
txt_name = "rain.txt"
csv_name = "rain_dirty.csv"

# colspecs and cols is formatting specific to this file
#the first col of the txt contains several pieces of information in a string
colspecs = [
    (0, 2), (2, 4), (4, 6), (6, 10),
    (10, 17), (17, 24), (24, 31), (31, 38),
    (38, 45), (45, 52), (52, 59), (59, 66),
    (66, 73), (73, 80), (80, 87), (87, 94)
]

cols = [
    "state", "division", "element", "year",
    "jan","feb","mar","apr","may","jun","jul","aug",
    "sep","oct","nov","dec"
]


# no input, reads txt file, converts strings to numeric values, writes csv with numeric values
def txt_to_csv(txt_name, csv_name, colspecs, cols):
    df = pd.read_fwf(txt_name, colspecs=colspecs, names=cols)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.to_csv(csv_name, index=False)


# takes a url as an input
#checks status code = 200 to read it
#takes data from website and writes to a txt file and a csv
# MUST BE A TXT FILE URL!!!
#read_url_txt(url, txt_name, csv_name, colspecs, cols)
def read_url_txt(url, txt_name, csv_name, colspecs, cols):
    # if url[-4] != ".txt":
    #     print("url must be a txt")
    r = requests.get(url)
    if r.status_code != 200:
        print(f"url status code is {r.status_code} not 200. Please check your url")
        return
    with open(txt_name, "wb") as f:
        f.write(r.content)
    txt_to_csv(txt_name, csv_name, colspecs, cols)

read_url_txt(url, txt_name, csv_name, colspecs, cols)

In [57]:
print(pd.read_csv(csv_name).head(1).to_markdown())

|    |   state |   division |   element |   year |   jan |   feb |   mar |   apr |   may |   jun |   jul |   aug |   sep |   oct |   nov |   dec |
|---:|--------:|-----------:|----------:|-------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
|  0 |       1 |          1 |         5 |   1895 |  0.11 | -0.81 | -0.56 | -0.72 | -0.85 | -0.99 |  -0.9 | -1.13 | -1.69 | -1.72 | -2.04 | -2.12 |


### Cleaning data
Make a new csv that contains normalized precip data by state

In [ ]:

months = ["jan","feb","mar",'apr',"may","jun","jul","aug","sep","oct","nov","dec"]
groups = ["state","year"]
new_col_name = "yearly_avg"
csv_name_clean = "rain_clean.csv"
df_to_read = "rain_dirty.csv"

# takes are precip data, combines the months, and divisions to get average rain per state per year. After I normalize it
def normalized_data(df_to_read, new_col_name, csv_name_clean, months, groups):
    #column with yearly average, combines months
    df = pd.read_csv(df_to_read)
    
    df[new_col_name] = df[months].mean(axis=1)
    
    # combines averages across divisions of the state. now average per state
    #then normalizes values across states and years (x-mean)/std
    
    state_precip = df.groupby(groups)[new_col_name].mean()
    #no need to normlaize, PDSI is already scaled by state
    #state_precip["stand_precip"] = (state_precip["avg_precip"] - state_precip["avg_precip"].mean()) / state_precip["avg_precip"].std()

    #writes df to csv
    state_precip.to_csv(csv_name_clean)


In [ ]:
normalized_data(df_to_read, new_col_name, csv_name_clean, months, groups)
print(pd.read_csv("rain_clean.csv").head(1).to_markdown())

|    |   state |   year |   yearly_avg |
|---:|--------:|-------:|-------------:|
|  0 |       1 |   1895 |    -0.821354 |


# Farm data - NOT USED.

In [ ]:
farm_data = pd.read_excel("VA_State_US.xlsx")
#farm_data.head()

In [ ]:
excel_path = "VA_State_US.xlsx"

# Read all sheets
all_sheets = pd.read_excel(excel_path, sheet_name=None)

all_state = pd.DataFrame()
# `all_sheets` is a dictionary: {sheet_name: DataFrame}



In [ ]:
print(all_sheets.items())

In [ ]:
for name, df in all_sheets.items():
    # print(f"Sheet: {name}")
    # print(df.head())
    print(name)
    if name != "Document Map":
        if name != "United States":
            df["state"] = name
            # removes rows where first column is NaN
            #df.drop([0,2,3,16,25,34,36,55,57,62,66,72,74,75,76])
            #df = df[df.iloc[:, 1].notna()]
            df.to_csv(f"state_farm_data/{name}.csv")

            all_state = pd.concat([all_state,df])

In [20]:
all_state.head()
all_state.to_csv("all_state.csv")

In [ ]:

combined_df = pd.concat(all_sheets.values(), ignore_index=True)
print(combined_df.head())
combined_df.to_csv("test.csv")


## Joining farm and precip data

In [50]:
full_farm_clean = pd.read_csv("FarmIncome_full.csv")

In [ ]:
#state_precip = pd.read_csv("rain_clean.csv")

csvs = ["FarmIncome_full.csv", "rain_clean.csv"]
group_on = ["state", "year"]
new_csv_name = "combined_farm_precip.csv"

def merge_csvs(csvs, group_on, new_csv_name):
    df1 = pd.read_csv(csvs[0])
    df2 = pd.read_csv(csvs[1])
    merged_df = pd.merge(df1, df2, on=group_on)
    merged_df.to_csv(new_csv_name)
    #return merged_df

merge_csvs(csvs, group_on, new_csv_name)

